In [1]:
import pandas as pd
import os
import re

# regular expression pattern to extract the URL

In [2]:
cwd = os.getcwd()
base_url = 'https://hsdes.intel.com/resource/'
pattern = r'"(.*?)"'

# Read the first CSV file into a Pandas DataFrame

In [3]:
HSD_data = pd.read_csv('.\data\MEV_SRS_HSD_WW09.csv')
DNG_data = pd.read_csv('.\data\MEV_SRS_DNG_WW09.csv')

# apply the regex to extract the URL and replace the original value in the column
need to extract ID from "=HYPERLINK("https://hsdes.intel.com/resource/14015250948","14015250948")"


In [4]:
HSD_data['H_url'] = HSD_data['id'].apply(lambda x: re.findall(pattern, x)[0])
HSD_data['H_Jama URL'] = base_url + HSD_data['id'].str.extract(r'(\d+)') 
HSD_data['Jama ID']  = HSD_data['id'].str.extract(r'(\d+)') 

HSD_data = HSD_data.drop('id', axis=1)
len(HSD_data)

130

In [5]:
HSD_data.iloc[1]

title         IPU Fuse Configuration Utility - tool description
component                                  sw.ie_tools_docs-ipu
H_url              https://hsdes.intel.com/resource/13010465679
H_Jama URL         https://hsdes.intel.com/resource/13010465679
Jama ID                                             13010465679
Name: 1, dtype: object

# prepeare DNG data

In [6]:
DNG_data['Jama ID']  = DNG_data['Jama ID'].str.extract(r'(\d+)') 

In [7]:
len(DNG_data)

681

In [8]:
DNG_data.iloc[1]

id                                                               377245
Name                                Document IMC Initialization Package
Artifact Type                                         ND SW Requirement
Jama ID                                                     22016633136
Jama URL              https://hsdes.intel.com/appstore/article/#/220...
HSD ID                                                              NaN
HSD URL                                                             NaN
Affected Component                                           Unassigned
Name: 1, dtype: object

# checking how many articles will be updated 

In [28]:
merged = pd.merge(HSD_data, DNG_data, on='Jama ID', how='inner')
len(merged)

124

# Create full list of articles to upload (update or insert)

In [29]:
merged = pd.merge(HSD_data, DNG_data, on='Jama ID', how='left')
len(merged)

130

In [30]:
merged.iloc[12]

title                                ACC UEFI FW - System Timer support
component                                       sw.pre-boot_acc_uefi_fw
H_url                      https://hsdes.intel.com/resource/18017059379
H_Jama URL                 https://hsdes.intel.com/resource/18017059379
Jama ID                                                     18017059379
id                                                               377135
Name                                 ACC UEFI FW - System Timer support
Artifact Type                                         ND SW Requirement
Jama URL              https://hsdes.intel.com/appstore/article/#/180...
HSD ID                                                              NaN
HSD URL                                                             NaN
Affected Component                                           Unassigned
Name: 12, dtype: object

In [31]:
merged = merged.drop('HSD ID', axis=1)
merged = merged.drop('HSD URL', axis=1)

merged = merged.drop('Name', axis=1)
merged = merged.rename(columns={'title': 'Name'})

merged = merged.drop('Affected Component', axis=1)
merged = merged.rename(columns={'component': 'Affected Component'})

merged = merged.drop('Jama URL', axis=1)
merged = merged.rename(columns={'H_Jama URL': 'Jama URL'})
merged = merged.rename(columns={'H_url': 'HSD URL'})

merged['HSD ID'] = merged['Jama ID']

merged['Artifact Type'] = 'ND SW Requirement'


In [32]:
merged.iloc[12]

Name                            ACC UEFI FW - System Timer support
Affected Component                         sw.pre-boot_acc_uefi_fw
HSD URL               https://hsdes.intel.com/resource/18017059379
Jama URL              https://hsdes.intel.com/resource/18017059379
Jama ID                                                18017059379
id                                                          377135
Artifact Type                                    ND SW Requirement
HSD ID                                                 18017059379
Name: 12, dtype: object

In [33]:
merged.to_csv('.\data\MEV_SRS_merge.csv', index=False)

In [5]:
#HSD_data.to_csv('.\data\WW09_HSD_CVL4.3_check.csv', index=False)